In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import itertools

* Sklearn

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score

Pytorch

In [3]:
import torch

from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import Sequential, Linear, SAGEConv, to_hetero
from torch.nn import ReLU
import torch.nn.functional as F

c:\Users\Daniel\anaconda3\envs\tfg_mlflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Paths and warnings

In [1]:
import time

In [3]:
aa = time.time()


In [6]:
time.time() - aa

4.028389930725098

In [4]:
import warnings
warnings.filterwarnings("ignore")

path = "../data/eda_generated_data/"
output_path = "../data/graph_data/"

In [5]:
from customer_inclusion import graph_test_no_prev, graph_test_prev
model = torch.load('../data/models/sage_heterogen_model.pt')
model_prev = torch.load('../data/models/sage_heterogen_model_prev.pt')

* Load data

In [6]:
def load_pickle(path, file_name):
    with open(path + file_name, 'rb') as f: return pickle.load(f)

path = "../data/graph_data/"
eda_path = "../data/eda_generated_data/"
output_path = "../data/models/"

In [7]:
training_graph = torch.load(path + 'training_graph.pt')
training_graph_prev = torch.load(path + 'training_graph_prev.pt')

df_test = load_pickle(eda_path, "df_test.pkl")

In [8]:
eda_path = "../data/eda_generated_data/"

graph_df = load_pickle(path, 'graph_df.pkl')
df_train = load_pickle(eda_path, 'df_train.pkl')
scaler = load_pickle(eda_path, 'scaler.pkl')

In [11]:
# training_graph_new_no_prev = graph_test_no_prev(df_test.iloc[:1000])
training_graph_new_prev = graph_test_prev(df_test.iloc[:1000])

In [10]:
training_graph_new_prev

HeteroData(
  users={
    x=[11000, 48],
    y=[10000],
    train_mask=[10000],
    valid_mask=[10000],
    test_mask=[11000]
  },
  previous={ x=[10492, 48] },
  (users, organization, users)={ edge_index=[2, 12780422] },
  (users, organization_weekday, users)={ edge_index=[2, 2011456] },
  (users, income_car_organization, users)={ edge_index=[2, 5304538] },
  (users, realty_organization, users)={ edge_index=[2, 7545006] },
  (users, suite_organization, users)={ edge_index=[2, 8626892] },
  (users, contract_organization, users)={ edge_index=[2, 10665314] },
  (users, housing_education_organization, users)={ edge_index=[2, 6293298] },
  (users, housing_family_organization, users)={ edge_index=[2, 4597520] },
  (users, has_previous, previous)={ edge_index=[2, 10492] },
  (previous, rev_has_previous, users)={ edge_index=[2, 10492] }
)

In [16]:
model_prev.eval()
out = model_prev(training_graph_new_prev.x_dict, training_graph_new_prev.edge_index_dict)
mask = training_graph_new_prev['users'].test_mask
pred = out['users'][mask].max(1)[1]

In [ ]:
model.eval()
out = model(training_graph_new_no_prev.x_dict, training_graph_new_no_prev.edge_index_dict)
mask = training_graph_new_no_prev['users'].test_mask
pred = out['users'][mask].max(1)[1]

In [17]:
roc_auc_score(df_test.iloc[:1000].TARGET, pred), f1_score(df_test.iloc[:1000].TARGET, pred), precision_score(df_test.iloc[:1000].TARGET, pred), recall_score(df_test.iloc[:1000].TARGET, pred)

(0.6429924242424243,
 0.2552083333333333,
 0.16554054054054054,
 0.5568181818181818)

In [14]:
roc_auc_score(df_test.iloc[:10000].TARGET, pred), f1_score(df_test.iloc[:10000].TARGET, pred), precision_score(df_test.iloc[:10000].TARGET, pred), recall_score(df_test.iloc[:10000].TARGET, pred)

(0.623787130028163,
 0.25165562913907286,
 0.18018967334035826,
 0.4170731707317073)